## Nested clustering of cells in the SCA, and cluster differential expression analysis (DEA)

In [4]:
import warnings
warnings.filterwarnings('ignore')

In this notebook we perform multi-level clustering of the cells in the SCA. We start with coarse-resolution clustering, and then re-cluster the resulting clusters by first re-calculating the nearest-neighbor graph and then clustering each cluster, with a finer resolution. For the final HLCA, we also calculate marker genes for every cluster. These will be used during manual annotation of the SCA.

In [5]:
import numpy as np
import pandas as pd
import scanpy as sc
import scanpy.external as sce
import sys
import os

sys.path.append("/fs/ess/PAS1475/guoqi/sci_atlas/Lung_splitdata/HLCA_reproducibility/scripts")
import nested_clustering
import analysis

In [6]:
path_SCA = "/fs/ess/PAS1475/guoqi/sci_atlas/Merge_health/health_scanvi_1117_sepleu.h5ad"
dir_benchmarking_cluster_output = "/fs/ess/PAS1475/guoqi/sci_atlas/Merge_health/Merge_Dec/Clustering/Benchmark/"
dir_SCA_cluster_output="/fs/ess/PAS1475/guoqi/sci_atlas/Merge_health/Merge_Dec/Clustering/"

In [7]:
dataset_name = "SCA"  # final, full HLCA

Set number of cluster levels (we used 3 for benchmark methods and for final atlas, and 4 for the full SCA)

In [8]:
number_of_clust_levels = 4

In [9]:
print("Dataset name:", dataset_name)
if dataset_name == "scanvi":
    # load dataset
    adata = sc.read(os.path.join(dir_benchmarking_res, "unscaled/hvg/scanvi.h5ad"))
    # specify which obsm to use for calculating neighbor graph
    use_rep = "X_emb"
    # specify whether or not to re-calculate the PCA for subsets of the object
    redo_pca = False
elif dataset_name == "seuratrpca":
    adata = sc.read(
        os.path.join(dir_benchmarking_res, "unscaled/hvg/R/seuratrpca.h5ad")
    )
    sc.tl.pca(adata)
    use_rep = "X_pca"
    redo_pca = True
elif dataset_name == "harmony":
    adata = sc.read(os.path.join(dir_bencmarking_res, "scaled/hvg/R/harmony.h5ad"))
    adata.obsm["X_pca"] = adata.obsm["X_emb"]
    use_rep = "X_emb"
    redo_pca = False
elif dataset_name == "SCA":
    adata = sc.read(path_SCA)
    use_rep = "X_scANVI"
    redo_pca = False

Dataset name: SCA


In [10]:
adata
use_rep = "X_scANVI"

In [11]:
for clustering_level in range(1, number_of_clust_levels + 1):
    print("clustering level:", clustering_level, "...")
    if clustering_level == 1:
        # skip for re-run
        cluster_name = "leiden_1"
        # first clustering is not nested, so use normal function:
        sc.pp.neighbors(adata, n_neighbors=30, use_rep=use_rep)
        sc.tl.leiden(adata, resolution=0.01, key_added=cluster_name)
    else:
        previous_clustering = "leiden_" + str(clustering_level - 1)
        cluster_name = "leiden_" + str(clustering_level)
        #         perform nested clustering
        #         set parameters:
        res = 0.2
        if clustering_level == 2:
            k = 30
            min_cluster_size = 50
        elif clustering_level == 3:
            k = 15
            min_cluster_size = 30
        elif clustering_level == 4:
            k = 10
            min_cluster_size = 10

        adata = nested_clustering.add_nested_clustering_blind(
            adata,
            previous_clustering,
            cluster_name,
            use_rep=use_rep,
            cluster_alg="leiden",
            cluster_k=k,
            cluster_res=res,
            min_cluster_size=min_cluster_size,
            redo_pca=redo_pca,  # SET THIS TO FALSE FOR SCANVI!!! OR OTHER EMBEDDING-OUTPUT METHODS!!!!!
        )
    # plot
    if "X_umap" in adata.obsm.keys():
        sc.pl.umap(adata, color=cluster_name)
    # store clustering:
    cluster_df = pd.DataFrame(adata.obs[cluster_name], index=adata.obs.index)
    # write to csv for benchmarking data:
    if dataset_name in ["harmony","scanvi","seuratrpca"]:
        cluster_df.to_csv(
            os.path.join(dir_benchmarking_cluster_output, f"{dataset_name}/{dataset_name}_{cluster_name}_cluster_assignment.csv")
        )
    # If wanted/needed, for final HLCA:
    if dataset_name == "SCA":
        # store cluster assignments:
        cluster_df.to_csv(
            os.path.join(dir_SCA_cluster_output, f"LCA_{cluster_name}_cluster_assignment.csv")
        )
        # calculate marker genes with respect to all other clusters, and with respect to sister clusters (i.e. other cluster from the same parent cluster):
        for marker_ref in ["sisters", "all"]:
            marker_gene_df = nested_clustering.get_cluster_markers(
                adata=adata,
                cluster_label=cluster_name,
                marker_ref=marker_ref,
                ngenes=100,
            )
            # and store:
            marker_gene_df.to_csv(
                os.path.join(dir_SCA_cluster_output, f"LCA_{cluster_name}_marker_genes_versus_{marker_ref}.csv")
            )

clustering level: 1 ...
Doing one versus sisters differential expression analysis.
ranking genes for parent group all
Doing one versus all differential expression analysis.
ranking genes for parent group all
clustering level: 2 ...
Not re-doing pca before nested clustering iterations!
Cluster: 0
reclustering...

calculating 30 nearest neighbors
using rep: X_scANVI
clustering
Cluster: 1
reclustering...

calculating 30 nearest neighbors
using rep: X_scANVI
clustering
Cluster: 2
reclustering...

calculating 30 nearest neighbors
using rep: X_scANVI
clustering
Cluster: 3
reclustering...

calculating 30 nearest neighbors
using rep: X_scANVI
clustering
Doing one versus sisters differential expression analysis.
ranking genes for parent group 0
ranking genes for parent group 1
ranking genes for parent group 2
ranking genes for parent group 3
Doing one versus all differential expression analysis.
ranking genes for parent group 0
ranking genes for parent group 1
ranking genes for parent group 2
r

In [21]:
cluster_name = "leiden_1"
# first clustering is not nested, so use normal function:
sc.pp.neighbors(adata, n_neighbors=30, use_rep=use_rep)
sc.tl.leiden(adata, resolution=0.01, key_added=cluster_name)

ValueError: Did not find X_scanvi_emb in `.obsm.keys()`. You need to compute it first.

In [28]:
adata.obsm["X_scANVI"]

array([[ 4.4126093e-02,  3.3352774e-01, -1.1815947e-01, ...,
        -2.2312836e-01, -3.4999669e-02, -1.2129725e+00],
       [-3.4622282e-01, -9.1891211e-01, -1.2108517e+00, ...,
        -5.7857841e-01, -2.5263126e+00,  1.0947158e+00],
       [-3.7217098e-01,  3.5347658e-01,  1.0170716e-01, ...,
        -6.3096121e-02,  4.4821414e-01, -7.4611187e-01],
       ...,
       [-6.6431171e-01,  4.6551228e-05,  9.0643823e-02, ...,
        -1.8450685e-01,  4.2656699e-01,  1.0230446e+00],
       [ 1.4800693e+00,  2.8973323e-01,  3.5206327e-01, ...,
        -7.8803003e-02,  7.3168504e-01, -6.3591421e-01],
       [ 4.3472809e-01,  9.1349244e-01,  6.6058671e-01, ...,
        -7.1263915e-01,  7.3647857e-02, -1.6532801e+00]], dtype=float32)

In [22]:
use_rep

'X_scanvi_emb'